In [33]:
import torch
import re 
from random import * 
import numpy as np 
import torch 
import torch.nn as nn 
import torch.optim as optim

In [215]:
import torch 

x = torch.Tensor([[1,2,3], [1,2,3], [1,2,3]])

b = 2
h = 3
l = 4
d = 5
r = 6 

x = torch.rand(b*h*l*d).view((b,h,l,d))
y = torch.rand(l*r*d).view((l,r,d))
print(y.size())
print(x.size())
# print(x)
ein_z = torch.einsum("bhld, lrd -> bhlr", x, y)
print("Einsum shape: ", ein_z.size())
# print(x.transpose(0,1))

x_reshaped = x.unsqueeze(3)
y_reshaped = y.unsqueeze(0).unsqueeze(1)
print(x_reshaped.shape)
print(y_reshaped.shape)
z = x_reshaped * y_reshaped
print("after mul: ", z.size())
z = torch.sum(z, dim=-1)
print("after sum: ", z.size())
# z = z.permute(b,h,l,r)
print(z.size())
z.allclose(ein_z)
# x.matmul(y)

# x_reshaped = x.reshape(b * h, l, d)
# y_reshaped = y.reshape(l, d, r)

# # Matrix multiplication
# z_reshaped = torch.matmul(x_reshaped, y_reshaped)
# print(z_reshaped.size())
# # Reshape z
# z = z_reshaped.reshape(b, h, l ,r)


# z.allclose(ein_z)

torch.Size([4, 6, 5])
torch.Size([2, 3, 4, 5])
Einsum shape:  torch.Size([2, 3, 4, 6])
torch.Size([2, 3, 4, 1, 5])
torch.Size([1, 1, 4, 6, 5])
after mul:  torch.Size([2, 3, 4, 6, 5])
after sum:  torch.Size([2, 3, 4, 6])
torch.Size([2, 3, 4, 6])


True

Relative positional encoding: https://jaketae.github.io/study/relative-positional-encoding/

In [194]:
a = torch.rand(2, 8, 3, 3)
b = torch.rand(2, 4, 3, 3)
ans = torch.matmul(a.unsqueeze(2), b.unsqueeze(1))

print("a unsqueeze size: ", a.unsqueeze(2).size())
print("b unsqueeze size: ", b.unsqueeze(1).size())

a unsqueeze size:  torch.Size([2, 8, 1, 3, 3])
b unsqueeze size:  torch.Size([2, 1, 4, 3, 3])


In [158]:
x = torch.Tensor([[[1,1],[2,2]], [[1,1],[2,2]]])
print(x.size())
torch.matmul(x,x)

torch.Size([2, 2, 2])


tensor([[[3., 3.],
         [6., 6.]],

        [[3., 3.],
         [6., 6.]]])

In [87]:
x = torch.Tensor([[[[1,2],[3,4]],[[5,6],[7,8]]],[[[9,10],[11,12]],[[13,14],[15,16]]]]) 
y = torch.Tensor([[[0,0],[0,0],[0,0]],[[0,0],[0,1],[0,0]]])
print("x size: ", x.size())
print("y size: ", y.size())

print("x: ", x)
print("y: ", y)

torch.einsum("bhld, lrd -> blhr", x, y)

a = torch.Tensor([[[1,2],[3,4]], [[1,2],[3,4]]])
b = torch.Tensor([[[1,2,3], [1,2,3]], [[1,2,4], [1,2,3]]])
c = torch.matmul(a,b)
print(c.size())
print(c)

x size:  torch.Size([2, 2, 2, 2])
y size:  torch.Size([2, 3, 2])
x:  tensor([[[[ 1.,  2.],
          [ 3.,  4.]],

         [[ 5.,  6.],
          [ 7.,  8.]]],


        [[[ 9., 10.],
          [11., 12.]],

         [[13., 14.],
          [15., 16.]]]])
y:  tensor([[[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 1.],
         [0., 0.]]])


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [2, 2] but got: [2, 1].

This code originated from here: https://towardsdatascience.com/deep-dive-into-the-code-of-bert-model-9f618472353e 

In [ ]:
from typing import Optional, Tuple

class BertSelfAttention(nn.Module): 
    def __init__(
        self, 
        num_attention_heads = 6, 
        hidden_size = 768,   
        dropout = 0.1,       
        position_embedding_type = "absolute", 

    ): 
        super().__init__()
        self.num_attention_heads = num_attention_heads
        self.attention_head_size = int(hidden_size / num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(hidden_size, self.all_head_size)
        self.key = nn.Linear(hidden_size, self.all_head_size)
        self.value = nn.Linear(hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(dropout)

    def forward(
        self, 
        hidden_states: torch.Tensor, 
        attention_mask: Optional[torch.FloatTensor] = None, 
        head_mask: Optional[torch.FloatTensor] = None, 
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None, 
        output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]: 
        mixed_query_layer = self.query(hidden_states)

        is_cross_attention = encoder_hidden_states is not None 

        if is_cross_attention and past_key_value is not None: 
            key_layer = past_key_value[0]
            value_layer = past_key_value[1]
            attention_mask = encoder_attention_mask
        elif is_cross_attention: 
            key_layer = self.transpose_for_scores(self.key(encoder_hidden_states))
            value_layer = self.transpose_for_scores(self.value(encoder_hidden_states))
            attention_mask = encoder_attention_mask
        elif past_key_value is not None: 
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))
            key_layer = torch.cat([past_key_value[0], key_layer], dim=2)
            value_layer = torch.cat([past_key_value[1], value_layer], dim=2)
        else: 
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))
        
        query_layer = self.transpose_for_scores(mixed_query_layer)



In [ ]:

class BertTokenizer(): 
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
        self, 
        vocab_file, 
        do_lower_case = True, 
        do_basic_tokenize= True,
        never_split=None,
        unk_token="[UNK]", 
        sep_token="[SEP]", 
        pad_token="[PAD]", 
        cls_token="[CLS]",
        mask_token="[MASK]", 
        tokenize_chinese_chars=True, 
        strip_accents=None,
        **kwargs
    ): 
        super().__init__(
            do_lower_case=do_lower_case, 
            do_basic_tokenize=do_basic_tokenize, 
            never_split=never_split, 
            unk_token=unk_token,
            sep_token=sep_token, 
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            tokenize_chinese_chars=tokenize_chinese_chars,
            strip_accents=strip_accents,
            **kwargs,
        )

        if not os.path.isfile(vocab_file): 
            raise ValueError(
                "cant find file"
            )

        self.vocab = load_vocab(vocab_file)
        self.ids_to_tokens = collections.OrderedDict([(ids, tok) for tok, ids in self.vocab.items()])
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab, unk_token=self.unk_token)

        output_tokens = []
        for token in whitespace_tokenize(text): 
            chars = list(token)
            if len(chars) > self.max_input_chars_per_word: 
                output_tokens.append(self.unk_token)
                continue 
        
            is_bad = False
            start = 0 
            sub_tokens = []
            while start < len(chars): 
                end = len(chars)
                cur_substr = None 
                while start < end: 
                    substr = "".join(chars[start:end])
                    if start > 0: 
                        substr = "##" + substr
                    if substr in self.vocab: 
                        cur_substr = substr
                        break
                    end -= 1 

                if cur_substr is None: 
                    is_bad = True 
                    break 

                sub_tokens.append(cur_substr)
                start = end 

            if is_bad: 
                output_tokens.append(self.unk_token)
            else: 
                output_tokens.extend(sub_tokens)
        return output_tokens 

In [31]:
from transformers import BertTokenizer, BertConfig, BertModel
import torch

max_length = 256
config = BertConfig.from_pretrained("bert-base-cased", output_attentions=True, output_hidden_states=True, return_dict=True)
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
config.max_position_embeddings = max_length

from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
inputs_tests = tokenizer(newsgroups_train['data'][:3], truncation=True, padding=True, max_length=max_length, return_tensors="pt")

model = BertModel(config)

inputs_tests


{'input_ids': tensor([[  101,  1622,   131,  5837,  1197,  1775,  2050,   137, 20049,  1306,
           119, 15276,  1181,   119,  5048,  1358,   113,  1187,   112,   188,
          1139,  1645,   114, 12859, 16811,   131,   160, 11612,  1942,  1610,
          1110,  1142,   106,   136,   151,  2227,  1643,   118,  3799,  1158,
           118, 15919,   131,   187,  7409,  1495,   119, 20049,  1306,   119,
         15276,  1181,   119,  5048,  1358,  6534,   131,  1239,  1104,  4624,
           117,  1531,  1670, 12058,   131,  1405,   146,  1108,  6123,  1191,
          2256,  1149,  1175,  1180,  4035,  4568,  1424,  1143,  1113,  1142,
          1610,   146,  1486,  1103,  1168,  1285,   119,  1135,  1108,   170,
           123,   118,  1442,  2865,  1610,   117,  1350,  1106,  1129,  1121,
          1103,  1523, 22891,   120,  1346, 19025,   119,  1135,  1108,  1270,
           170, 19217,  2836,   119,  1109,  3581,  1127,  1541,  1353,   119,
          1130,  1901,   117,  1103,  

In [32]:
n_batch = 3
shape_embs = (inputs_tests['input_ids'].shape) + (model.embeddings.word_embeddings.weight.shape[1], )
w_embs_batch = torch.index_select(model.embeddings.word_embeddings.weight, 0, inputs_tests['input_ids'].reshape(1,-1).squeeze(0)).reshape(shape_embs)
pos_embs_batch = torch.index_select(model.embeddings.position_embeddings.weight, 0, 
                                    torch.tensor(range(inputs_tests['input_ids'][1].size(0))).repeat(1, n_batch).squeeze(0)).reshape(shape_embs)
type_embs_batch = torch.index_select(model.embeddings.token_type_embeddings.weight, 0, 
                                     inputs_tests['token_type_ids'].reshape(1,-1).squeeze(0)).reshape(shape_embs)
batch_all_embs = w_embs_batch + pos_embs_batch + type_embs_batch
batch_all_embs.shape # (batch_size, n_words, embedding dim)


torch.Size([3, 256, 768])

In [ ]:
class BertModel(nn.Module): 
    def __init__(self): 
        super().__init__()
        self.embeddings = BertEmbeddings() 
        self.encoder = BertEncoder()
        self.pooler = BertPooler() if add_pooling_layer else None 
    
    def forward(
        self, 
        input_ids: Optional[torch.Tensor] = None, 
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None, 
        encoder_hidden_states: Optional[torch.Tensor] = None, 
        past_key_values: Optional[List[torch.FloatTensor]] = None, 
        use_cache: Optional[bool] = None, 
        output_attentions: Optional[bool] = None, 
        output_hidden_states: Optional[bool] = None, 
        return_dict: Optional[bool] = None 
    ):
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions 
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.use_return_dict
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

In [36]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
inputs_tests = tokenizer(newsgroups_train['data'][:3], truncation=True, padding=True, max_length=max_length)

In [ ]:
f1 = torch.index_select(model.embeddings.word_embeddings.weight, 0, inputs_tests['input_ids'][0]) \
+ torch.index_select(model.embeddings.position_embeddings.weight, 0, torch.tensor(range(inputs_tests['input_ids'][0].size(0))).long()) \
+ torch.index_select(moedl.embeddings.token_type_embeddings.weight, 0, inputs_tests["token_type_ids"][0])

In [43]:
tokenizer.tokenize("hello world yo. my name is jy jeff")

['hello',
 'world',
 'yo',
 '.',
 'my',
 'name',
 'is',
 'j',
 '##y',
 'j',
 '##ef',
 '##f']

In [40]:
inputs_tests['attention_mask'][0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
from sklearn.datasets import fetch_20newsgroups 
newsgroups_train = fetch_20newsgroups(subset='train')

max_length = 256

inputs_tests = tokenizer(newsgroups_train['data'][:3], truncation=True, padding=True, max_length=max_length, return_tensors='pt')



In [4]:
text = (
        'Hello, how are you? I am Romeo.\n'
        'Hello, Romeo My name is Juliet. Nice to meet you.\n'
        'Nice meet you too. How are you today?\n'
        'Great. My baseball team won the competition.\n'
        'Oh Congratulations, Juliet\n'
        'Thanks you Romeo'
    )

In [7]:
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')
word_list = list(set(" ".join(sentences).split()))
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

for i, w in enumerate(word_list): 
    word_dict[w] = i + 4
number_dict = {i: w for i, w in enumerate(word_dict)}
vocab_size = len(word_dict)

token_list = list()
for sentence in sentences: 
    arr = [word_dict[s] for s in sentence.split()]
    token_list.append(arr)

In [8]:
token_list

[[19, 28, 10, 22, 14, 9, 26],
 [19, 26, 25, 21, 6, 15, 23, 18, 8, 22],
 [23, 8, 22, 13, 28, 10, 22, 16],
 [7, 25, 4, 12, 11, 20, 5],
 [24, 27, 15],
 [17, 22, 26]]

In [9]:
maxlen = 30 
batch_size = 6 
max_pred = 5 
n_layers = 6 
n_heads = 12
d_model = 768 
d_ff = 768 
d_k = d_v = 64 
n_segments = 2

In [ ]:
def make_batch(): 
    batch = []
    positive = negative = 0 
    while positive != batch_size/2 or negative != batch_size/2: 
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange(len(sentences))
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]
        

In [ ]:
class BERT(nn.Module): 
    def __init__(self): 
        super(BERT, self).__init__()
        self.embedding = Embedding() 
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu 
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        embed_weight = self.embedding.tok_embed.weight 
        n_vocab, n_dim = embed_weight.size() 
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))
    
    def forward(self, input_ids, segment_ids, masked_pos): 
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers: 
            output, enc_self_attn = layer(output, enc_self_attn_mask)

        h_pooled = self.activ1(self.fc(output[:, 0]))
        logits_clsf = self.classifier(h_pooled)

        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1))
        h_masked = torch.gather(output, 1, masked_pos)
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias

        return logits_lm, logits_clsf 
